# Goal of this Lecture Note

In these notes, we want to investigate the role of the *learning rate*, in particular its magnitude, in the gradient descent algorithm, starting with the simplest possible setting.

---

## Setup

Consider the general formulation of gradient descent.  
Let $\beta \in \mathbb{R}^d$ be the parameters we want to estimate, and let $\mathcal{L}(\beta)$ be the objective (e.g., loss function) we want to minimize.  
Gradient descent updates the parameters according to

$$
\hat{\beta}_{t+1} = \hat{\beta}_t - \lambda \nabla_{\beta} \mathcal{L}(\hat{\beta}_t),
$$

where $\lambda > 0$ is the **learning rate**.

---

## Why the Magnitude of the Learning Rate Matters

The learning rate controls the fundamental trade-off between **speed** and **stability**.

The gradient provides the *direction* of steepest descent, but the learning rate determines the *distance* traveled along that direction at each step.

- If **$\lambda$** is *small*, each update is cautious. Convergence is stable but slow.
- If **$\lambda$** is *large but still below the stability threshold*, updates are aggressive and convergence is fast.
- If **$\lambda$** is *too large*, the algorithm overshoots the minimum, oscillates, and can eventually diverge.

To study these behaviors precisely and transparently, we now turn to a simple model where everything can be computed explicitly: **linear regression with an intercept and slope**.

---

## A Simple Test Case: Linear Regression

We consider the parameter vector

$$
\beta =
\begin{pmatrix}
\beta^{(0)} \\
\beta^{(1)}
\end{pmatrix}
\in \mathbb{R}^2,
$$

and the (mean squared error) loss function

$$
\mathcal{L}(\beta)
=
\frac{1}{N}
\sum_{i=1}^N \big( y_i - \beta^{(0)} - \beta^{(1)} x_i \big)^2,
$$

where the dataset is

$$
\mathcal{D} = \{(x_i, y_i)\}_{i=1}^N.
$$

This setting is simple enough to allow us to derive the gradient, Hessian, and the exact dynamical system generated by gradient descent, making it ideal for studying the role of the learning rate.

## Calculating the gradient

Then the gradient of the loss is
$$
\begin{align}
\nabla_\beta \mathcal{L}(\beta) = 
-2\begin{pmatrix}
\bar{y}-\beta^{(0)} - \beta^{(1)}\bar{x}\\
\overline{yx}- \beta^{(0)} \bar{x} - \beta^{(1)} \overline{x^2}
\end{pmatrix}
\end{align},
$$
where 
$$
\bar{x} = \frac{1}{N}\sum_{i=1}^N x_i, \quad
\bar{y} = \frac{1}{N}\sum_{i=1}^N y_i, \quad
\overline{x^2} = \frac{1}{N}\sum_{i=1}^N x_i^2, \quad
\overline{xy} = \frac{1}{N}\sum_{i=1}^N x_i y_i.
$$

The dynamical system that describes the gradient descent is
$$
\begin{pmatrix}
\hat{\beta}^{(0)}_{t+1}\\[4pt]
\hat{\beta}^{(1)}_{t+1}
\end{pmatrix}
=
\begin{pmatrix}
\hat{\beta}^{(0)}_{t}\\[4pt]
\hat{\beta}^{(1)}_{t}
\end{pmatrix}
+ \lambda\times 2
\begin{pmatrix}
\bar{y}-\hat{\beta}^{(0)}_t - \hat{\beta}^{(1)}_t \bar{x}\\[6pt]
\overline{xy}- \hat{\beta}^{(0)}_t \bar{x} - \hat{\beta}^{(1)}_t \overline{x^2}
\end{pmatrix},
$$

which simplifies to
$$
\hat{\beta}_{t+1} = (I-2\lambda P)\hat{\beta}_t + 2\lambda c
$$

where 

$$
P \equiv \begin{bmatrix}
1 & \bar{x}\\
\bar{x} & \overline{x^2}
\end{bmatrix}, 
~\text{and}~
c \equiv \begin{pmatrix}
\bar{y}\\
\overline{xy}
\end{pmatrix}
$$

## Fixed point
This system has a unique fixed point that solves
$$
P \beta^* = b ~ \text{or}~ \begin{bmatrix}
1 & \bar{x}\\
\bar{x} & \overline{x^2}
\end{bmatrix} \begin{pmatrix}
\beta^{(0)^*}\\
\beta^{(1)^*} 
\end{pmatrix} = \begin{pmatrix}
\bar{y}\\
\overline{xy}
\end{pmatrix}
$$
Therefore
$$
\begin{align}
\beta^* = P^{-1}c
\end{align}
$$
This is exactly the 2-D version of $(X'X)^{-1}X'Y$ that we learn in stats/econometrics.

## The stability of the dynamical system 
Define 
$$
\hat{e}_t \equiv \hat{\beta}_t-\beta^*,
$$
which represents the deviation from the fixed point. The dynamics of this deviation are
$$
\begin{align}
\hat{e}_{t+1} = (I-2\lambda P) \hat{e}_t.
\end{align}
$$

Let $\mu_1$ and $\mu_2$ be the eigenvalues of $P$, and let $\nu_1$ and $\nu_2$ be the corresponding eigenvectors.

The solution can be written as 

$$
\begin{align}
\hat{e}_t = \alpha_1 \nu_1 (1-2\lambda \mu_1)^t + \alpha_2 \nu_2 (1-2\lambda \mu_2)^t 
\end{align}
$$
where $\alpha_1$ and $\alpha_2$ are determined by the initial condition $e_0 = \hat{\beta}_0-\beta^*$. 

### Convergence
For gradient descent to converge, we need
$$
\lim_{t\rightarrow 0} \hat{e}_t = 0 \quad \text{for all} \quad \hat{e}_0
$$

That means 
1. $|1-2\lambda \mu_1|<1$ $\quad$ $\rightarrow$ $\quad$ $0<\lambda<\frac{1}{\mu_1}$
2. $|1-2\lambda \mu_2|<1$ $\quad$ $\rightarrow$ $\quad$ $0<\lambda<\frac{1}{\mu_2}$

Therefore
$$
\begin{align}
\boxed{0 < \lambda < \frac{1}{\mu_{\text{max}}}}
\end{align}
$$

-----
Let’s test this in practice. Here I use `PyTorch`, but the gradient can also be defined manually to reproduce the same update rule.

In [13]:
## Importing packages
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
## Generating data
torch.manual_seed(1)
N = 200
true_b0 = 1.5
true_b1 = -2.0

x = torch.randn(N, 1)                   # shape (N, 1)
eps = 0.1 * torch.randn(N, 1)
y = true_b0 + true_b1 * x + eps 

In [66]:
## Forming P matrix
x_bar = x.mean()
x_sq_bar = (x**2).mean()

P = torch.tensor([[1.0,x_bar],[x_bar,x_sq_bar]])

eigenvalues = torch.linalg.eigvalsh(P)

μ_max = eigenvalues.max().item()

In [78]:
## define the linear function (neural net)
model = nn.Linear(in_features=1, out_features=1)

with torch.no_grad():
    model.weight.fill_(0.0)   # sets w = 0
    model.bias.fill_(0.0)     # sets b = 0


## setting the optimizer
learning_rate = (1/μ_max)*(1.01)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [84]:
b0_list_diverge = []
b1_list_diverge = []

num_steps = 100

for t in range(num_steps):

    # Forward pass
    y_pred = model(x)

    # Compute loss
    loss = criterion(y_pred, y)

    # Zero gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Gradient descent update
    optimizer.step()

    # Extract parameters
    b0_hat = model.bias.item()
    b1_hat = model.weight.item()

    # Save them
    b0_list_diverge.append(b0_hat)
    b1_list_diverge.append(b1_hat)

    # Optional printout
    #print(f"step {t:4d} | loss = {loss:.5f} | b0 = {b0_hat:.3f}, b1 = {b1_hat:.3f}")


step    0 | loss = 888188.62500 | b0 = 316.228, b1 = -877.056
step    1 | loss = 924072.00000 | b0 = -319.534, b1 = 890.536
step    2 | loss = 961404.25000 | b0 = 328.943, b1 = -912.408
step    3 | loss = 1000244.50000 | b0 = -332.503, b1 = 926.595
step    4 | loss = 1040654.56250 | b0 = 342.172, b1 = -949.187
step    5 | loss = 1082695.25000 | b0 = -345.996, b1 = 964.109
step    6 | loss = 1126435.62500 | b0 = 355.935, b1 = -987.453
step    7 | loss = 1171943.50000 | b0 = -360.035, b1 = 1003.140
step    8 | loss = 1219289.25000 | b0 = 370.254, b1 = -1027.264
step    9 | loss = 1268548.37500 | b0 = -374.641, b1 = 1043.748
step   10 | loss = 1319796.75000 | b0 = 385.152, b1 = -1068.684
step   11 | loss = 1373115.62500 | b0 = -389.836, b1 = 1085.996
step   12 | loss = 1428590.25000 | b0 = 400.652, b1 = -1111.777
step   13 | loss = 1486302.50000 | b0 = -405.646, b1 = 1129.950
step   14 | loss = 1546347.87500 | b0 = 416.777, b1 = -1156.611
step   15 | loss = 1608820.62500 | b0 = -422.094, 